In [38]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error

In [39]:
df=pd.read_csv('House Price Prediction Dataset.csv')
df.shape
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         2000 non-null   int64 
 1   Area       2000 non-null   int64 
 2   Bedrooms   2000 non-null   int64 
 3   Bathrooms  2000 non-null   int64 
 4   Floors     2000 non-null   int64 
 5   YearBuilt  2000 non-null   int64 
 6   Location   2000 non-null   object
 7   Condition  2000 non-null   object
 8   Garage     2000 non-null   object
 9   Price      2000 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 156.4+ KB


,Id,Area,Bedrooms,Bathrooms,Floors,YearBuilt,Location,Condition,Garage,Price
0,1,1360,5,4,3,1970,Downtown,Excellent,No,149919
1,2,4272,5,4,3,1958,Downtown,Excellent,No,424998
2,3,3592,2,2,3,1938,Downtown,Good,No,266746
3,4,966,4,2,2,1902,Suburban,Fair,Yes,244020
4,5,4926,1,4,2,1975,Downtown,Fair,Yes,636056
5,6,3944,1,2,1,1906,Urban,Poor,No,93262
6,7,3671,1,1,2,1948,Rural,Poor,Yes,448722
7,8,3419,2,4,1,1925,Suburban,Good,Yes,594893
8,9,630,2,2,1,1932,Rural,Poor,Yes,652878
9,10,2185,3,3,1,2000,Downtown,Poor,No,340375


In [40]:
df['Area']=df['Area'].fillna(df['Area'].mean())
df['Bedrooms']=df['Bedrooms'].fillna(df['Bedrooms'].mean())
df['Price']=df['Price'].fillna(df['Price'].mean())
df['Location']=df['Location'].fillna(df['Location'].mode()[0])
df=pd.get_dummies(df,drop_first=True)
df=df.apply(pd.to_numeric,errors='coerce')
scaler=StandardScaler()
df[['Area','Bedrooms']]=scaler.fit_transform(df[['Area','Bedrooms']])
df=df.astype(float)
X=df.drop('Price',axis=1).values
y=df['Price'].values
X_b=np.hstack([np.ones((X.shape[0],1)), X])
beta=np.linalg.pinv(X_b.T @ X_b)@(X_b.T@y)
def predict(x):
    x=np.hstack([np.ones((x.shape[0],1)), x])
    return x@beta
y_pred=predict(X)
print("RMSE ",np.sqrt(mean_squared_error(y,y_pred)))
print("R2 ",r2_score(y,y_pred))

RMSE  275133.849599504
R2  0.008851950897529548


In [41]:
kf=KFold(n_splits=5,shuffle=True,random_state=1)
rmse=[]
r2=[]
for train_idx,test_idx in kf.split(X):
    X_train,X_test=X[train_idx],X[test_idx]
    y_train,y_test=y[train_idx],y[test_idx]
    X_train_b=np.hstack([np.ones((X_train.shape[0],1)),X_train])
    X_test_b=np.hstack([np.ones((X_test.shape[0],1)),X_test])
    beta_fold=np.linalg.pinv(X_train_b.T@X_train_b)@(X_train_b.T@y_train)
    y_pred_fold=X_test_b@beta_fold
    rmse.append(np.sqrt(mean_squared_error(y_test,y_pred_fold)))
    r2.append(r2_score(y_test,y_pred_fold))
print("\n5-Fold Cross Validation:")
print("RMSE (each fold):", rmse)
print("Average RMSE:", np.mean(rmse))
print("\nR² (each fold):", r2)
print("Average R²:", np.mean(r2))


5-Fold Cross Validation:
RMSE (each fold): [np.float64(278276.6951583792), np.float64(277096.5274004148), np.float64(275424.43268855725), np.float64(283213.15338529146), np.float64(273928.6257561729)]
Average RMSE: 277587.8868777632

R² (each fold): [-0.009486412343614026, -0.014684858453887228, -0.019405819117645606, -0.01366804072571437, -0.007144983636866664]
Average R²: -0.012878022855545578
